<a href="https://colab.research.google.com/github/vidush5/Advanced-NLP-Techniques/blob/master/Movie_Review_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

In [2]:
print(tf.__version__)

2.2.0


In [0]:
import os
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import numpy as np

In [6]:
datasets = tfds.load(name='imdb_reviews', split=('train[:70%]', 'train[70%:85%]', 'train[85%:]'), as_supervised=True)


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVYW2F4/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVYW2F4/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVYW2F4/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
train_data=datasets[0]
test_data=datasets[1]
validation_data=datasets[2]

In [22]:
type(train_data)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

In [0]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [24]:
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [25]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [0]:
pretrained_model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(pretrained_model, input_shape=[], dtype=tf.string, trainable=True)

In [27]:
train_examples_batch[:2]

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell a

In [28]:
hub_layer(train_examples_batch[:2])

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ]],
      dtype=float32)>

In [29]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_4 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [31]:
model.fit(train_data.shuffle(10000).batch(512),
          epochs = 20,
          validation_data = validation_data.batch(512),
          verbose=1)

Epoch 1/20
35/35 [==============================] - 2s 65ms/step - loss: 0.6542 - accuracy: 0.6157 - val_loss: 0.6115 - val_accuracy: 0.6619
Epoch 2/20
35/35 [==============================] - 2s 61ms/step - loss: 0.5920 - accuracy: 0.6857 - val_loss: 0.5645 - val_accuracy: 0.7171
Epoch 3/20
35/35 [==============================] - 2s 61ms/step - loss: 0.5465 - accuracy: 0.7297 - val_loss: 0.5226 - val_accuracy: 0.7581
Epoch 4/20
35/35 [==============================] - 2s 62ms/step - loss: 0.5024 - accuracy: 0.7643 - val_loss: 0.4831 - val_accuracy: 0.7824
Epoch 5/20
35/35 [==============================] - 2s 61ms/step - loss: 0.4562 - accuracy: 0.7958 - val_loss: 0.4427 - val_accuracy: 0.8075
Epoch 6/20
35/35 [==============================] - 2s 62ms/step - loss: 0.4108 - accuracy: 0.8230 - val_loss: 0.4054 - val_accuracy: 0.8269
Epoch 7/20
35/35 [==============================] - 2s 61ms/step - loss: 0.3684 - accuracy: 0.8487 - val_loss: 0.3753 - val_accuracy: 0.8405
Epoch 8/20
35

In [32]:
model.predict(["This is the worst movie I have ever seen"])

array([[0.09419637]], dtype=float32)

In [33]:
model.predict(["An excellent movie that I enjoyed a lot"])

array([[0.99830073]], dtype=float32)